In [1]:
class RNNConfig():
    input_size = 1
    num_steps = 252
    lstm_size = 128
    num_layers = 1
    keep_prob = 0.8

    batch_size = 64
    init_learning_rate = 0.001
    learning_rate_decay = 0.99
    init_epoch = 5
    max_epoch = 50

    def to_dict(self):
        dct = self.__class__.__dict__
        return {k: v for k, v in dct.items() if not k.startswith('__') and not callable(v)}

    def __str__(self):
        return str(self.to_dict())

    def __repr__(self):
        return str(self.to_dict())


DEFAULT_CONFIG = RNNConfig()
print ("Default configuration:", DEFAULT_CONFIG.to_dict())

DATA_DIR = "data"
LOG_DIR = "logs"
MODEL_DIR = "models"

Default configuration: {'input_size': 1, 'num_steps': 252, 'lstm_size': 128, 'num_layers': 1, 'keep_prob': 0.8, 'batch_size': 64, 'init_learning_rate': 0.001, 'learning_rate_decay': 0.99, 'init_epoch': 5, 'max_epoch': 50}


In [2]:
import numpy as np
import random
import tensorflow as tf

def build_lstm_graph_with_config(config=None):
    tf.reset_default_graph()
    lstm_graph = tf.Graph()

    if config is None:
        config = DEFAULT_CONFIG

    with lstm_graph.as_default():
        """
        The model asks for three things:
        - input: training data X
        - targets: training label y
        - learning_rate:
        """
        learning_rate = tf.placeholder(tf.float32, None, name="learning_rate")

        # Number of examples, number of input, dimension of each input
        inputs = tf.placeholder(tf.float32, [None, config.num_steps, config.input_size], name="inputs")
        targets = tf.placeholder(tf.float32, [None, config.input_size], name="targets")

        def _create_one_cell():
            lstm_cell = tf.contrib.rnn.LSTMCell(config.lstm_size, state_is_tuple=True)
            if config.keep_prob < 1.0:
                lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=config.keep_prob)
            return lstm_cell

        cell = tf.contrib.rnn.MultiRNNCell(
            [_create_one_cell() for _ in range(config.num_layers)],
            state_is_tuple=True
        ) if config.num_layers > 1 else _create_one_cell()

        val, _ = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32, scope="lstm_rnn")

        # Before transpose, val.get_shape() = (batch_size, num_steps, lstm_size)
        # After transpose, val.get_shape() = (num_steps, batch_size, lstm_size)
        val = tf.transpose(val, [1, 0, 2])

        with tf.name_scope("output_layer"):
            # last.get_shape() = (batch_size, lstm_size)
            last = tf.gather(val, int(val.get_shape()[0]) - 1, name="last_lstm_output")

            weight = tf.Variable(tf.truncated_normal([config.lstm_size, config.input_size]), name="lstm_weights")
            bias = tf.Variable(tf.constant(0.1, shape=[config.input_size]), name="lstm_biases")
            prediction = tf.matmul(last, weight) + bias

            tf.summary.histogram("last_lstm_output", last)
            tf.summary.histogram("weights", weight)
            tf.summary.histogram("biases", bias)

        with tf.name_scope("train"):
            # loss = -tf.reduce_sum(targets * tf.log(tf.clip_by_value(prediction, 1e-10, 1.0)))
            loss = tf.reduce_mean(tf.square(prediction - targets), name="loss_mse")
            optimizer = tf.train.AdamOptimizer(learning_rate)
            minimize = optimizer.minimize(loss, name="loss_mse_adam_minimize")
            tf.summary.scalar("loss_mse", loss)

        # Operators to use after restoring the model
        for op in [prediction, loss]:
            tf.add_to_collection('ops_to_restore', op)

    return lstm_graph

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passi

In [5]:
import json
import os
import sys; sys.path.append("..")

from data_model import StockDataSet


def load_data(stock_name, input_size, num_steps):
    stock_dataset = StockDataSet(stock_name, input_size=input_size, num_steps=num_steps,
                                 test_ratio=0.2)
    print ("Train data size:", len(stock_dataset.train_X))
    print ("Test data size:", len(stock_dataset.test_X))
    return stock_dataset


def _compute_learning_rates(config=DEFAULT_CONFIG):
    learning_rates_to_use = [
        config.init_learning_rate * (
            config.learning_rate_decay ** max(float(i + 1 - config.init_epoch), 0.0)
        ) for i in range(config.max_epoch)
    ]
    print ("Middle learning rate:", learning_rates_to_use[len(learning_rates_to_use) // 2])
    return learning_rates_to_use


def train_lstm_graph(stock_name, lstm_graph, config=DEFAULT_CONFIG):
    """
    stock_name (str)
    lstm_graph (tf.Graph)
    """
    stock_dataset = load_data(stock_name,
                              input_size=config.input_size,
                              num_steps=config.num_steps)

    final_prediction = []
    final_loss = None

    graph_name = "%s_lr%.2f_lr_decay%.3f_lstm%d_step%d_input%d_batch%d_epoch%d" % (
        stock_name,
        config.init_learning_rate, config.learning_rate_decay,
        config.lstm_size, config.num_steps,
        config.input_size, config.batch_size, config.max_epoch)

    print ("Graph Name:", graph_name)

    learning_rates_to_use = _compute_learning_rates(config)
    with tf.Session(graph=lstm_graph) as sess:
        merged_summary = tf.summary.merge_all()
        writer = tf.summary.FileWriter('_logs/' + graph_name, sess.graph)
        writer.add_graph(sess.graph)

        graph = tf.get_default_graph()
        tf.global_variables_initializer().run()

        inputs = graph.get_tensor_by_name('inputs:0')
        targets = graph.get_tensor_by_name('targets:0')
        learning_rate = graph.get_tensor_by_name('learning_rate:0')

        test_data_feed = {
            inputs: stock_dataset.test_X,
            targets: stock_dataset.test_y,
            learning_rate: 0.0
        }

        loss = graph.get_tensor_by_name('train/loss_mse:0')
        minimize = graph.get_operation_by_name('train/loss_mse_adam_minimize')
        prediction = graph.get_tensor_by_name('output_layer/add:0')

        for epoch_step in range(config.max_epoch):
            current_lr = learning_rates_to_use[epoch_step]

            for batch_X, batch_y in stock_dataset.generate_one_epoch(config.batch_size):
                train_data_feed = {
                    inputs: batch_X,
                    targets: batch_y,
                    learning_rate: current_lr
                }
                train_loss, _ = sess.run([loss, minimize], train_data_feed)

            if epoch_step % 1 == 0:
                test_loss, _pred, _summary = sess.run([loss, prediction, merged_summary], test_data_feed)
                assert len(_pred) == len(stock_dataset.test_y)
                print ("Epoch %d [%f]:" % (epoch_step, current_lr), test_loss)
                if epoch_step % 50 == 0:
                    print ("Predictions:", [(
                        map(lambda x: round(x, 4), _pred[-j]),
                        map(lambda x: round(x, 4), stock_dataset.test_y[-j])
                    ) for j in range(5)])

            writer.add_summary(_summary, global_step=epoch_step)

        print ("Final Results:")
        final_prediction, final_loss = sess.run([prediction, loss], test_data_feed)
        print (final_prediction, final_loss)

        graph_saver_dir = os.path.join(MODEL_DIR, graph_name)
        if not os.path.exists(graph_saver_dir):
            os.mkdir(graph_saver_dir)

        saver = tf.train.Saver()
        saver.save(sess, os.path.join(
            graph_saver_dir, "stock_rnn_model_%s.ckpt" % graph_name), global_step=epoch_step)

    with open("final_predictions.{}.json".format(graph_name), 'w') as fout:
        fout.write(json.dumps(final_prediction.tolist()))


def main(config=DEFAULT_CONFIG):
    lstm_graph = build_lstm_graph_with_config(config=config)
    train_lstm_graph('BCOMTR', lstm_graph, config=config)


In [6]:
graph1 = build_lstm_graph_with_config()
train_lstm_graph('BCOMTR',graph1)

Train data size: 3695
Test data size: 924
Graph Name: BCOMTR_lr0.00_lr_decay0.990_lstm128_step252_input1_batch64_epoch50
Middle learning rate: 0.0008097278682212584
Epoch 0 [0.001000]: 9.582439e-05
Predictions: [(<map object at 0x19ac70ef0>, <map object at 0x19ac70c18>), (<map object at 0x19ac70e48>, <map object at 0x19ac70eb8>), (<map object at 0x19ac70f98>, <map object at 0x19ac70cc0>), (<map object at 0x19ac70c88>, <map object at 0x19ac70dd8>), (<map object at 0x1990a7fd0>, <map object at 0x1990a7c18>)]
Epoch 1 [0.001000]: 0.00010537681
Epoch 2 [0.001000]: 0.00010100955
Epoch 3 [0.001000]: 9.006514e-05
Epoch 4 [0.001000]: 9.097673e-05
Epoch 5 [0.000990]: 0.00010000819
Epoch 6 [0.000980]: 8.3556726e-05
Epoch 7 [0.000970]: 0.00010351632
Epoch 8 [0.000961]: 9.02151e-05
Epoch 9 [0.000951]: 8.960639e-05
Epoch 10 [0.000941]: 8.815577e-05
Epoch 11 [0.000932]: 8.0000595e-05
Epoch 12 [0.000923]: 9.146439e-05
Epoch 13 [0.000914]: 8.084805e-05
Epoch 14 [0.000904]: 8.194542e-05
Epoch 15 [0.0008

In [37]:
def prediction_by_trained_graph(graph_name, max_epoch, test_X, test_y):
    test_prediction = None
    test_loss = None
    result = []
    
    with tf.Session() as sess:
        # Load meta graph
        graph_meta_path = os.path.join(
            MODEL_DIR, graph_name,
            'stock_rnn_model_{0}.ckpt-{1}.meta'.format(graph_name, max_epoch-1))
        checkpoint_path = os.path.join(MODEL_DIR, graph_name)

        saver = tf.train.import_meta_graph(graph_meta_path)
        saver.restore(sess, tf.train.latest_checkpoint(checkpoint_path))

        graph = tf.get_default_graph()

        for i in range(0,len(test_X)//30-1):
            r = []
            for j in range(0,30):
        
                test_feed_dict = {
                    graph.get_tensor_by_name('inputs:0'): [test_X[i*30]],
                    graph.get_tensor_by_name('targets:0'): [test_y[0]],
                    graph.get_tensor_by_name('learning_rate:0'): 0.0
                }

                prediction = graph.get_tensor_by_name('output_layer/add:0')
                loss = graph.get_tensor_by_name('train/loss_mse:0')
                test_prediction, test_loss = sess.run([prediction, loss], test_feed_dict)
                curr = test_prediction
                r.append(curr[0][0])
                a = test_X[i*30][1:]
                temp = np.concatenate((a,curr))
                test_X[i*30] = temp
            
            result.append(r)
     
    return result

In [38]:
data_1 = load_data('BCOMTR',1,252)
len(data_1.test_X)

Train data size: 3695
Test data size: 924


924

In [39]:
graph_name = 'BCOMTR_lr0.00_lr_decay0.990_lstm128_step252_input1_batch64_epoch50'
curr = prediction_by_trained_graph(graph_name,50,data_1.test_X,data_1.test_y)
curr

INFO:tensorflow:Restoring parameters from models/BCOMTR_lr0.00_lr_decay0.990_lstm128_step252_input1_batch64_epoch50/stock_rnn_model_BCOMTR_lr0.00_lr_decay0.990_lstm128_step252_input1_batch64_epoch50.ckpt-49


[[0.00053166226,
  0.0016395804,
  0.00068954565,
  0.0014435742,
  -0.004381096,
  -0.0023123175,
  0.00024988875,
  -0.0029000882,
  -0.0017789546,
  -0.00321991,
  0.0010036733,
  -0.0020931084,
  -0.0017941669,
  0.00311872,
  -0.0016927756,
  0.0015525613,
  0.003521338,
  0.00010590628,
  -0.0009426549,
  -0.0014500096,
  -0.003625257,
  -0.0021385942,
  -0.00013753399,
  -0.0020121615,
  0.0011012256,
  0.0061659925,
  -0.004909439,
  -0.0029528122,
  4.548952e-05,
  0.0027175862],
 [0.0013503768,
  -0.004222773,
  -0.00010679662,
  0.0037978757,
  0.000100594014,
  -0.0011410303,
  0.001241006,
  0.0056973323,
  -0.00063841976,
  -0.0046127345,
  0.00067911297,
  0.0010841303,
  -0.0033800192,
  0.00011305511,
  -0.002183957,
  0.0007727649,
  -0.0015830938,
  -0.0008686986,
  -0.0020658895,
  0.0065067243,
  -0.001792511,
  -0.0033051,
  0.00046649575,
  0.0008479599,
  0.003477499,
  5.353242e-05,
  -0.0003777556,
  -0.004761521,
  0.0017636195,
  0.0022306293],
 [0.005600836

In [57]:
import pandas as pd
df = pd.DataFrame(curr).transpose()
df = df.apply(lambda x:x+1)
df

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,1.000532,1.001350,1.005601,1.003441,1.000070,0.994872,1.002932,1.003162,0.999103,0.994075,...,0.999161,1.000604,0.998971,0.999604,1.000882,1.001512,1.000704,0.998002,1.001364,1.005253
1,1.001640,0.995777,1.000838,0.999359,1.000361,1.003105,1.001167,1.000239,1.001104,1.004370,...,1.008060,0.999071,0.999040,1.003164,1.000697,1.004472,0.996947,1.000818,0.996905,0.998745
2,1.000690,0.999893,1.000243,0.997915,1.003306,0.999713,0.999585,1.000854,1.000730,1.001095,...,0.998274,0.996688,1.000536,0.995517,1.001557,1.000904,0.998192,0.999494,1.001637,0.998750
3,1.001444,1.003798,0.997297,1.000892,0.995788,0.998871,0.996032,0.999731,0.996866,0.997907,...,1.004761,0.996095,0.999654,0.998574,1.000842,1.001208,0.995979,1.002457,0.997040,1.001010
4,0.995619,1.000101,1.002404,1.002533,0.999131,0.995654,1.000047,1.002212,0.999342,0.996800,...,0.998116,0.996855,1.002280,0.997804,1.004492,0.997356,1.002129,1.002706,1.002721,1.003468
5,0.997688,0.998859,1.002493,0.997750,0.998784,1.000803,0.998941,1.001195,1.003502,0.999981,...,0.999929,1.001267,1.001664,0.998648,0.996947,0.994957,0.995813,0.995965,0.998684,0.999919
6,1.000250,1.001241,0.998212,0.998280,1.001449,1.000059,1.001523,1.001367,1.000325,0.997391,...,0.997765,0.999815,1.000437,1.001267,1.001270,0.998225,0.998510,0.999647,0.998140,0.999065
7,0.997100,1.005697,0.999503,1.003671,1.001617,0.998845,1.001713,1.002482,1.000035,1.000686,...,0.997595,1.007221,0.997454,0.998715,0.999303,0.999486,0.998214,0.999120,0.999911,0.997387
8,0.998221,0.999362,1.004710,0.996494,1.001066,1.001344,1.001578,0.999478,1.001510,0.997280,...,0.998410,0.999652,0.993186,1.000584,0.998019,0.996629,1.001803,1.005218,1.005848,1.002940
9,0.996780,0.995387,0.996326,0.996393,0.999574,1.002638,0.999571,1.001599,0.999296,0.998120,...,1.004484,0.999893,0.997677,1.000857,0.997249,1.001508,1.001225,1.002705,0.998851,0.997325


In [42]:
def load_ori_data(stock_name, input_size, num_steps):
    stock_dataset = StockDataSet(stock_name, input_size=input_size, num_steps=num_steps,
                                 test_ratio=0.2, normalized = False)
    print ("Train data size:", len(stock_dataset.train_X))
    print ("Test data size:", len(stock_dataset.test_X))
    return stock_dataset
data_1_ori = load_ori_data('BCOMTR',1,252)

Train data size: 3695
Test data size: 924


In [46]:
# format
data_1_ori.test_X[0][-1][0]

167.7598

In [58]:
for i in range(len(df.iloc[0])):
    temp = data_1_ori.test_X[i*30][-1][0]
    for j in range(len(df)):
        df[i][j] = df[i][j]*temp
        temp = df[i][j]

df.to_csv('1.csv')

In [29]:
graph1 = build_lstm_graph_with_config()
for ind in ['RU20INTR','RUMCINTR','LBUSTRUU','NDDUWXUS','LG30TRUU']:
    train_lstm_graph(ind,graph1)

Train data size: 3695
Test data size: 924
Graph Name: RU20INTR_lr0.00_lr_decay0.990_lstm128_step252_input1_batch64_epoch50
Middle learning rate: 0.0008097278682212584
Epoch 0 [0.001000]: 0.0002820059
Predictions: [(<map object at 0x257544fd0>, <map object at 0x257544f98>), (<map object at 0x257565128>, <map object at 0x257565048>), (<map object at 0x257565080>, <map object at 0x257565208>), (<map object at 0x257565278>, <map object at 0x257565390>), (<map object at 0x257565320>, <map object at 0x2575653c8>)]
Epoch 1 [0.001000]: 0.0002884073
Epoch 2 [0.001000]: 0.00030568778
Epoch 3 [0.001000]: 0.00029508522
Epoch 4 [0.001000]: 0.00030183163
Epoch 5 [0.000990]: 0.00029169765
Epoch 6 [0.000980]: 0.00028358639
Epoch 7 [0.000970]: 0.00027460878
Epoch 8 [0.000961]: 0.0002934897
Epoch 9 [0.000951]: 0.00031780254
Epoch 10 [0.000941]: 0.00029558782
Epoch 11 [0.000932]: 0.0002781977
Epoch 12 [0.000923]: 0.00028779023
Epoch 13 [0.000914]: 0.00027716276
Epoch 14 [0.000904]: 0.00027901898
Epoch 15

Train data size: 3695
Test data size: 924
Graph Name: RUMCINTR_lr0.00_lr_decay0.990_lstm128_step252_input1_batch64_epoch50
Middle learning rate: 0.0008097278682212584
Epoch 0 [0.001000]: 0.0002273008
Predictions: [(<map object at 0x255f82240>, <map object at 0x255f82358>), (<map object at 0x255f82b38>, <map object at 0x255f82710>), (<map object at 0x255f82860>, <map object at 0x255f82898>), (<map object at 0x255f82940>, <map object at 0x255f82550>), (<map object at 0x255f82470>, <map object at 0x255f82748>)]
Epoch 1 [0.001000]: 0.00021081461
Epoch 2 [0.001000]: 0.00022537487
Epoch 3 [0.001000]: 0.00022882497
Epoch 4 [0.001000]: 0.0002213052
Epoch 5 [0.000990]: 0.00020797964
Epoch 6 [0.000980]: 0.0002177709
Epoch 7 [0.000970]: 0.00022004222
Epoch 8 [0.000961]: 0.00022870227
Epoch 9 [0.000951]: 0.00021277645
Epoch 10 [0.000941]: 0.00022064344
Epoch 11 [0.000932]: 0.00021653896
Epoch 12 [0.000923]: 0.00021303949
Epoch 13 [0.000914]: 0.0002054739
Epoch 14 [0.000904]: 0.00021085501
Epoch 15

Train data size: 3695
Test data size: 924
Graph Name: LBUSTRUU_lr0.00_lr_decay0.990_lstm128_step252_input1_batch64_epoch50
Middle learning rate: 0.0008097278682212584
Epoch 0 [0.001000]: 3.7002374e-05
Predictions: [(<map object at 0x257544e48>, <map object at 0x257565240>), (<map object at 0x2575651d0>, <map object at 0x19a2a6780>), (<map object at 0x19a2a69b0>, <map object at 0x19a2a6d30>), (<map object at 0x19a2a69e8>, <map object at 0x19a2a6cc0>), (<map object at 0x19a2a6cf8>, <map object at 0x19a2a6668>)]
Epoch 1 [0.001000]: 3.119561e-05
Epoch 2 [0.001000]: 2.7916285e-05
Epoch 3 [0.001000]: 2.8918883e-05
Epoch 4 [0.001000]: 2.7159356e-05
Epoch 5 [0.000990]: 2.6714466e-05
Epoch 6 [0.000980]: 2.9901747e-05
Epoch 7 [0.000970]: 3.0758583e-05
Epoch 8 [0.000961]: 2.2702956e-05
Epoch 9 [0.000951]: 2.8987008e-05
Epoch 10 [0.000941]: 2.0946558e-05
Epoch 11 [0.000932]: 1.935832e-05
Epoch 12 [0.000923]: 2.6506246e-05
Epoch 13 [0.000914]: 1.9898012e-05
Epoch 14 [0.000904]: 2.4271549e-05
Epoch 

Train data size: 3695
Test data size: 924
Graph Name: NDDUWXUS_lr0.00_lr_decay0.990_lstm128_step252_input1_batch64_epoch50
Middle learning rate: 0.0008097278682212584
Epoch 0 [0.001000]: 0.00012327374
Predictions: [(<map object at 0x257544e80>, <map object at 0x257544e48>), (<map object at 0x19a2a6e10>, <map object at 0x19a2a6eb8>), (<map object at 0x199b6cf60>, <map object at 0x199b6cdd8>), (<map object at 0x199b6cef0>, <map object at 0x199b6c860>), (<map object at 0x199b6cc18>, <map object at 0x199b6cf28>)]
Epoch 1 [0.001000]: 0.0001334754
Epoch 2 [0.001000]: 0.00011911109
Epoch 3 [0.001000]: 0.00014444083
Epoch 4 [0.001000]: 0.000119984295
Epoch 5 [0.000990]: 0.00011459658
Epoch 6 [0.000980]: 0.000120154786
Epoch 7 [0.000970]: 0.00012733761
Epoch 8 [0.000961]: 0.00012890446
Epoch 9 [0.000951]: 0.000120797406
Epoch 10 [0.000941]: 0.00010668199
Epoch 11 [0.000932]: 0.00011169673
Epoch 12 [0.000923]: 0.000112793874
Epoch 13 [0.000914]: 0.00011372575
Epoch 14 [0.000904]: 0.00011508857
E

Train data size: 3695
Test data size: 924
Graph Name: LG30TRUU_lr0.00_lr_decay0.990_lstm128_step252_input1_batch64_epoch50
Middle learning rate: 0.0008097278682212584
Epoch 0 [0.001000]: 4.4660912e-05
Predictions: [(<map object at 0x255f82e10>, <map object at 0x255f82f60>), (<map object at 0x199c3be48>, <map object at 0x1999e53c8>), (<map object at 0x1999e5940>, <map object at 0x1999e5278>), (<map object at 0x1999e5860>, <map object at 0x1999e5438>), (<map object at 0x1999e5668>, <map object at 0x1999e52b0>)]
Epoch 1 [0.001000]: 3.8756978e-05
Epoch 2 [0.001000]: 3.6403697e-05
Epoch 3 [0.001000]: 3.5021632e-05
Epoch 4 [0.001000]: 3.8121947e-05
Epoch 5 [0.000990]: 3.3090924e-05
Epoch 6 [0.000980]: 3.0779218e-05
Epoch 7 [0.000970]: 3.2447628e-05
Epoch 8 [0.000961]: 2.8602568e-05
Epoch 9 [0.000951]: 3.0413905e-05
Epoch 10 [0.000941]: 3.0120198e-05
Epoch 11 [0.000932]: 2.641705e-05
Epoch 12 [0.000923]: 2.8260469e-05
Epoch 13 [0.000914]: 2.9392259e-05
Epoch 14 [0.000904]: 2.691949e-05
Epoch 

In [59]:
for ind in ['RU20INTR','RUMCINTR','LBUSTRUU','NDDUWXUS','LG30TRUU']:
    graph_name = 'BCOMTR_lr0.00_lr_decay0.990_lstm128_step252_input1_batch64_epoch50'
    data_ = load_data(ind,1,252)
    data_ori = load_ori_data(ind,1,252)
    curr = prediction_by_trained_graph(graph_name,50,data_.test_X,data_.test_y)
    df = pd.DataFrame(curr).transpose()
    df = df.apply(lambda x:x+1)
    for i in range(len(df.iloc[0])):
        temp = data_ori.test_X[i*30][-1][0]
        for j in range(len(df)):
            df[i][j] = df[i][j]*temp
            temp = df[i][j]

    df.to_csv('%s.csv'%ind)

Train data size: 3695
Test data size: 924
Train data size: 3695
Test data size: 924
INFO:tensorflow:Restoring parameters from models/BCOMTR_lr0.00_lr_decay0.990_lstm128_step252_input1_batch64_epoch50/stock_rnn_model_BCOMTR_lr0.00_lr_decay0.990_lstm128_step252_input1_batch64_epoch50.ckpt-49
Train data size: 3695
Test data size: 924
Train data size: 3695
Test data size: 924
INFO:tensorflow:Restoring parameters from models/BCOMTR_lr0.00_lr_decay0.990_lstm128_step252_input1_batch64_epoch50/stock_rnn_model_BCOMTR_lr0.00_lr_decay0.990_lstm128_step252_input1_batch64_epoch50.ckpt-49
Train data size: 3695
Test data size: 924
Train data size: 3695
Test data size: 924
INFO:tensorflow:Restoring parameters from models/BCOMTR_lr0.00_lr_decay0.990_lstm128_step252_input1_batch64_epoch50/stock_rnn_model_BCOMTR_lr0.00_lr_decay0.990_lstm128_step252_input1_batch64_epoch50.ckpt-49
Train data size: 3695
Test data size: 924
Train data size: 3695
Test data size: 924
INFO:tensorflow:Restoring parameters from m

SyntaxError: invalid syntax (<ipython-input-30-771318ce8650>, line 1)